[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepmipt/RDLS_NLP_2022/blob/dev/lectures/04.Creating_Dialog_Systems/seminar/notebooks/01.basics.ipynb)

In [ ]:
# !pip install df_engine df_runner df_db_connector df_telegram_connector

<img src="https://raw.githubusercontent.com/kudep/RDLS_images/bba12c082827415f7d3e1ab09dfb94d68415be93/dfe_examples/script_structire.svg" >
<img src="https://raw.githubusercontent.com/kudep/RDLS_images/bba12c082827415f7d3e1ab09dfb94d68415be93/dfe_examples/node_sructire.svg" >

## Short description
First of all, to create a dialog agent, we need to create a dialog script.
Below, `script` is the dialog script.
A dialog script is a flow dictionary that can contain multiple script .
Script are needed in order to divide a dialog into sub-dialogs and process them separately.
For example, the separation can be tied to the topic of the dialog.
In our example, there is one flow called greeting_flow.

Inside each flow, we can describe a sub-dialog.
Here we can also use keyword `LOCAL`, which we have considered in other examples.

Flow describes a sub-dialog using linked nodes, each node has the keywords `RESPONSE` and `TRANSITIONS`.

`RESPONSE` - contains the response that the dialog agent will return when transitioning to this node.
`TRANSITIONS` - describes transitions from the current node to other nodes.
`TRANSITIONS` are described in pairs:
     - the node to which the agent will perform the transition
     - the condition under which to make the transition

In [ ]:
# imports
from df_engine.core.keywords import TRANSITIONS, RESPONSE
import df_engine.conditions as cnd

# script init
script = {
    "greeting_flow": {
        "start_node": {  # This is an initial node, it doesn't need an `RESPONSE`
            RESPONSE: "",
            TRANSITIONS: {"node1": cnd.exact_match("Hi")},  # If "Hi" == request of user then we make the transition
        },
        "node1": {
            RESPONSE: "Hi, how are you?",  # When the agent goes to node1, we return "Hi, how are you?"
            TRANSITIONS: {"node2": cnd.exact_match("i'm fine, how are you?")},
        },
        "node2": {
            RESPONSE: "Good. What do you want to talk about?",
            TRANSITIONS: {"node3": cnd.exact_match("Let's talk about music.")},
        },
        "node3": {
            RESPONSE: "Sorry, I can not talk about music now.",
            TRANSITIONS: {"node4": cnd.exact_match("Ok, goodbye.")},
        },
        "node4": {RESPONSE: "bye", TRANSITIONS: {"node1": cnd.exact_match("Hi")}},
        "fallback_node": {  # We get to this node if an error occurred while the agent was running
            RESPONSE: "Ooops",
            TRANSITIONS: {"node1": cnd.exact_match("Hi")},
        },
    }
}

<center><img src="https://raw.githubusercontent.com/kudep/RDLS_images/ae2d4ef5645e17e9cd28ba9341da0242c786024a/dfe_examples/example_basics.svg" width="400" height="400"> </center>

In [ ]:
from df_runner import ScriptRunner

runner = ScriptRunner(
    script,
    start_label=("greeting_flow", "start_node"),
    fallback_label=("greeting_flow", "fallback_node"),
)

In [ ]:
runner.start()